In [1]:
# Installing required packages
%pip install -r requirements.txt
%pip install --upgrade --no-index --find-links=wheelhouse heapcy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: wheelhouse
Processing ./wheelhouse/heapcy-0.1.6-cp313-cp313-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl
  Attempting uninstall: heapcy
    Found existing installation: heapcy 0.1.5
    Uninstalling heapcy-0.1.5:
      Successfully uninstalled heapcy-0.1.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
size_of_heap = 1_000
import random
def gen_random_string(le=12)->str:
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789~!@#$%^&*-_=+;:\\`"
    random.seed()
    pwd_len= random.randrange(1,le)
    prob = random.uniform(0,1)
    final_str= ""
    for _ in range(pwd_len):
        char= random.randrange(le)
        final_str += alphabet[char]
    return final_str + " "+str(prob)+"\n"

In [3]:
import tempfile
def create_temp_file_for_benchmark()->str:
    tf = tempfile.NamedTemporaryFile(delete=False)
    lis = [ gen_random_string() for _ in range(size_of_heap)]
    with open(tf.name,"w") as f_open:
        f_open.writelines(lis)
    return tf.name
    

In [4]:
from heapitem import HeapItem
from pympler import asizeof
import heapq
heap = []
t = create_temp_file_for_benchmark()
with open(t,"rb") as f_open:
    while True:
        line = f_open.readline()
        if not line:
            break
        parts = line.split(b" ",1)
        
        heapq.heappush(heap,HeapItem(float(parts[1]),parts[0].decode(encoding="ascii")))
    
print(f"sizeof normal heap with heapitem = {asizeof.asizeof(heap)/1000/1000}MB")
del heap


sizeof normal heap with heapitem = 0.051632MB


In [5]:
import heapcy,os
heap = heapcy.Heap(size_of_heap)
tf = create_temp_file_for_benchmark()
with open(tf,"rb") as f_open:

    while True:
        offset = f_open.tell()
        line = f_open.readline()
        if not line:
            break

        parts = line.split(b" ", 1)
        if len(parts) != 2:
            continue

        prob = float(parts[1].decode(encoding="ascii"))
        if len(heap)< size_of_heap:
            heapcy.heappush(heap,prob,offset)
        elif heapcy.heappeek(heap)[0]>prob:
            heapcy.heappoppush(heap,prob,offset)

print(f"sizeof special heap = {asizeof.asizeof(heap)/1000/1000}MB")
del heap

sizeof special heap = 0.016056MB


In [6]:
import heapq 
heap = []

with open(tf,"rb") as f_open:
     while True:
        line = f_open.readline()
        if not line:
            break
        parts = line.split(b" ")
        if len(parts) != 2:
            continue

        prob = float(parts[1].decode(encoding="ascii"))

        if len(heap)< size_of_heap:
            heapq.heappush(heap,(prob,parts[0]))
        elif heap[0][0]>prob:
            heapq.heappoppush(heap,(prob,parts[0]))

print(f"sizeof  heapq = {asizeof.asizeof(heap)/1000/1000}MB")
del heap
os.remove(tf)

sizeof  heapq = 0.1286MB
